# Streamgraph en la R.M.
> Streamgraph en la R.M.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/fallecimientos.png

In [70]:
#hide
import numpy as np
import pandas as pd
import altair as alt

In [139]:
#hide
county_cumulative_cases = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19.csv")

In [140]:
#hide
county_cumulative_cases_RM = county_cumulative_cases.query("Region=='Metropolitana'").drop(
    columns=["Region", "Codigo region", "Poblacion", "Tasa", "Codigo comuna"])

In [141]:
#hide
county_cases_RM = county_cumulative_cases_RM.set_index("Comuna").T.diff().drop("2020-03-30").fillna(0)

In [142]:
county_cases_RM["Total"] = county_cases_RM.sum(axis=1)

In [143]:
#hide
county_cases_RM["Total"] = county_cases_RM.sum(axis=1)
county_cases_RM = county_cases_RM.T.apply(lambda x: x/x[-1]).T

In [144]:
#hide
start_date = county_cases_RM.reset_index()["index"].iloc[0]
end_date = county_cases_RM.reset_index()["index"].iloc[-1]

data = pd.DataFrame()
data['Fecha'] = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d')
data = data.set_index("Fecha")

In [146]:
#hide
data[list(county_cases_RM.columns)] = county_cases_RM[list(county_cases_RM.columns)]

In [147]:
#hide
data = data.interpolate(method="linear")

In [148]:
#hide
last_known_proportion = data.iloc[-1]

In [149]:
#hide
regional_cumulative_cases = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv')

In [150]:
#hide
regional_cumulative_cases_RM = regional_cumulative_cases.query("Region=='Metropolitana'").iloc[-1].drop("Region")
regional_cases_RM = regional_cumulative_cases_RM.diff().dropna()

In [151]:
#hide
last_known_date = county_cases_RM.index[-1]
last_known_date

'2020-07-10'

In [152]:
#hide
n_missing_days = (pd.to_datetime(regional_cases_RM.index[-1])-pd.to_datetime(data.index[-1])).days

In [153]:
#hide
data = data.T

In [154]:
#hide
for i in np.arange(n_missing_days):
    idx = (pd.to_datetime(last_known_date)+pd.DateOffset(1+i)).strftime("%Y-%m-%d")
    print(idx)
    data[idx] = last_known_proportion

2020-07-11
2020-07-12
2020-07-13


In [155]:
#hide
data = data.T

In [156]:
#hide
data["Regional"] = regional_cases_RM

In [157]:
#hide
outlier = data.loc["2020-06-17","Regional"]
outlier

32230

In [158]:
#hide
data.loc["2020-06-17","Regional"] = data.loc["2020-06-16","Regional"]

In [159]:
#hide
total = data.loc[:"2020-06-17", "Regional"].sum()
total

150904

In [160]:
#hide
proportion = data.loc[:"2020-06-17", "Regional"]/total
proportion.tail(2)

Fecha
2020-06-16    0.0266527
2020-06-17    0.0266527
Name: Regional, dtype: object

In [161]:
#hide
data.loc[:"2020-06-17", "Regional"] = data.loc[:"2020-06-17", "Regional"] + (proportion*outlier).astype(int)

In [162]:
#hide
data = data.T.apply(lambda x: x[:-1]*x[-1]).T.astype(int)

In [163]:
#hide
list_verde = ['La Reina', 'Las Condes', 'Lo Barnechea', 'Nunoa',  'Providencia', 'Vitacura']
list_rojo = ['Santiago', 'Conchali', 'Huechuraba', 'Independencia', 'Recoleta', 'Quilicura',
            'Cerro Navia', 'Lo Prado', 'Pudahuel', 'Quinta Normal', 'Renca',
            'Estacion Central', 'Maipu', 'Cerrillos', 'Padre Hurtado', 'Penaflor']
list_azul = ['Pedro Aguirre Cerda', 'San Miguel', 'San Joaquin', 'Lo Espejo', 'La Cisterna', \
           'La Granja', 'San Ramon', 'El Bosque', 'San Bernardo', 'La Pintana',
            "Macul", "La Florida", 'Penalolen', "Puente Alto"]

In [164]:
#hide
cases_verde = data[list_verde]
cases_rojo = data[list_rojo]
cases_azul = data[list_azul]

Mapas de [Juanizio Correa](https://twitter.com/Juanizio_C):

> twitter: https://twitter.com/PolarBearby/status/1265826899198017536

In [165]:
#hide
# data = pd.DataFrame()
# data["Verde"] = np.round(cases_verde.sum(axis=1).rolling(window=7).mean())
# data["Rojo"] = np.round(cases_rojo.sum(axis=1).rolling(window=7).mean())
# data["Azul"] = np.round(cases_azul.sum(axis=1).rolling(window=7).mean())

In [166]:
#hide
data = pd.DataFrame()
data["Verde"] = cases_verde.sum(axis=1)
data["Rojo"] = cases_rojo.sum(axis=1)
data["Azul"] = cases_azul.sum(axis=1)

In [167]:
#hide
data_total = data.sum(axis=1)

In [168]:
#hide
data_percentage = pd.DataFrame()
data_percentage["Verde"] = np.round(100*data["Verde"]/data_total, decimals=1)
data_percentage["Rojo"] = np.round(100*data["Rojo"]/data_total, decimals=1)
data_percentage["Azul"] = np.round(100*data["Azul"]/data_total, decimals=1)

In [169]:
#hide
data = pd.DataFrame()
data["Verde"] = np.round(cases_verde.sum(axis=1).rolling(window=7).mean())
data["Rojo"] = np.round(cases_rojo.sum(axis=1).rolling(window=7).mean())
data["Azul"] = np.round(cases_azul.sum(axis=1).rolling(window=7).mean())

In [170]:
#hide
data = data.reset_index().melt("Fecha", var_name="Sector", value_name="Casos confirmados")

In [171]:
#hide
data_percentage = data_percentage.reset_index().melt("Fecha", var_name="Sector", value_name="Porcentaje")

In [172]:
#hide_input

domain = ["Azul", "Rojo", "Verde"]
range_ = ["blue", "red", "green"]

chart1 = alt.Chart(data).mark_area().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False)),
    y=alt.Y("Casos confirmados", axis=None, stack="center"),
    tooltip = ["Fecha", "Casos confirmados"],
    color = alt.Color("Sector", scale=alt.Scale(domain=domain, range=range_)),
).properties(
    title="Nuevos casos confirmados en la R.M. por zona socioeconómica (promedio móvil 7 días)",
    width=700)

chart2 = alt.Chart(data_percentage).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, ticks=False, labels=False)),
    y=alt.Y("Porcentaje", axis=alt.Axis(title="", format='%'), stack="normalize"),
    tooltip = ["Fecha", "Porcentaje"],
    color = alt.Color("Sector"),
).properties(
    height = 100,
    width=700
)

alt.vconcat(
    chart1,
    chart2
).configure_concat(
    spacing=0
)

alt.VConcatChart(...)

Fuentes: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19), [Ministerio de Salud](https://www.minsal.cl/)

In [173]:
#hide
county_cumulative_cases = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19.csv")

In [174]:
#hide
county_cumulative_cases_RM = county_cumulative_cases.query("Region=='Metropolitana'").drop(
    columns=["Region", "Codigo region", "Poblacion", "Tasa", "Codigo comuna"])

In [175]:
#hide
county_cases_RM = county_cumulative_cases_RM.set_index("Comuna").T.diff().drop("2020-03-30").fillna(0)

In [176]:
#hide
county_cases_RM["Total"] = county_cases_RM.sum(axis=1)
county_cases_RM = county_cases_RM.T.apply(lambda x: x/x[-1]).T

In [177]:
#hide
start_date = county_cases_RM.reset_index()["index"].iloc[0]
end_date = county_cases_RM.reset_index()["index"].iloc[-1]

data = pd.DataFrame()
data['Fecha'] = pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d')
data = data.set_index("Fecha")

In [178]:
#hide
data[list(county_cases_RM.columns)] = county_cases_RM[list(county_cases_RM.columns)]

In [179]:
#hide
data = data.interpolate(method="linear")

In [180]:
#hide
last_known_proportion = data.iloc[-1]

In [181]:
#hide
regional_cumulative_cases = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv')

In [182]:
#hide
regional_cumulative_cases_RM = regional_cumulative_cases.query("Region=='Metropolitana'").iloc[-1].drop("Region")
regional_cases_RM = regional_cumulative_cases_RM.diff().dropna()

In [183]:
#hide
last_known_date = county_cases_RM.index[-1]
last_known_date

'2020-07-10'

In [184]:
#hide
n_missing_days = (pd.to_datetime(regional_cases_RM.index[-1])-pd.to_datetime(data.index[-1])).days

In [185]:
#hide
data = data.T

In [186]:
#hide
for i in np.arange(n_missing_days):
    idx = (pd.to_datetime(last_known_date)+pd.DateOffset(1+i)).strftime("%Y-%m-%d")
    print(idx)
    data[idx] = last_known_proportion

2020-07-11
2020-07-12
2020-07-13


In [187]:
#hide
data = data.T

In [188]:
#hide
data["Regional"] = regional_cases_RM

In [189]:
#hide
outlier = data.loc["2020-06-17","Regional"]
outlier

32230

In [190]:
#hide
data.loc["2020-06-17","Regional"] = data.loc["2020-06-16","Regional"]

In [191]:
#hide
total = data.loc[:"2020-06-17", "Regional"].sum()
total

150904

In [192]:
#hide
proportion = data.loc[:"2020-06-17", "Regional"]/total
proportion.tail(2)

Fecha
2020-06-16    0.0266527
2020-06-17    0.0266527
Name: Regional, dtype: object

In [193]:
#hide
data.loc[:"2020-06-17", "Regional"] = data.loc[:"2020-06-17", "Regional"] + (proportion*outlier).astype(int)

In [194]:
#hide
data = data.T.apply(lambda x: x[:-1]*x[-1]).T.astype(int)

In [195]:
#hide
list_nororiente = ['La Reina', 'Las Condes', 'Lo Barnechea', 'Nunoa',  'Providencia', 'Vitacura'] #
list_suroriente = ["Macul", "La Florida", 'Penalolen', "Puente Alto"] #
list_norponiente = ['Cerro Navia', 'Lo Prado', 'Pudahuel', 'Quinta Normal', 'Renca'] #
list_surponiente = ['Estacion Central', 'Maipu', 'Cerrillos', 'Padre Hurtado', 'Penaflor'] #
list_norte = ['Conchali', 'Huechuraba', 'Independencia', 'Recoleta', 'Quilicura'] #
list_centro = ['Santiago'] #
list_sur = ['Pedro Aguirre Cerda', 'San Miguel', 'San Joaquin', 'Lo Espejo', 'La Cisterna', \
           'La Granja', 'San Ramon', 'El Bosque', 'San Bernardo', 'La Pintana'] #

In [196]:
#hide
cases_nororiente = data[list_nororiente]
cases_suroriente = data[list_suroriente]
cases_norponiente = data[list_norponiente]
cases_surponiente = data[list_surponiente]
cases_norte = data[list_norte]
cases_centro = data[list_centro]
cases_sur = data[list_sur]

In [197]:
#hide
data = pd.DataFrame()
data["7. Nororiente"] = cases_nororiente.sum(axis=1).rolling(window=7).mean()
data["6. Suroriente"] = cases_suroriente.sum(axis=1).rolling(window=7).mean()
data["5. Norte"] = cases_norte.sum(axis=1).rolling(window=7).mean()
data["4. Centro"] = cases_centro.sum(axis=1).rolling(window=7).mean()
data["3. Sur"] = cases_sur.sum(axis=1).rolling(window=7).mean()
data["2. Norponiente"] = cases_norponiente.sum(axis=1).rolling(window=7).mean()
data["1. Surponiente"] = cases_surponiente.sum(axis=1).rolling(window=7).mean()

In [198]:
#hide
data = data.reset_index()

In [199]:
#hide
data = data.melt("Fecha", var_name="Sector", value_name="Casos confirmados")

In [200]:
#hide_input
chart1 = alt.Chart(data).mark_area().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False)),
    y=alt.Y("Casos confirmados", axis=None, stack="center"),
    tooltip = ["Fecha", "Casos confirmados"],
    color = alt.Color("Sector"),
).properties(
    title="Nuevos casos confirmados en la R.M. por zona geográfica (promedio móvil 7 días)",
    width=700)

chart2 = alt.Chart(data).mark_bar().encode(
    x=alt.X("Fecha", axis=alt.Axis(title="", domain=False, ticks=False, labels=False)),
    y=alt.Y("Casos confirmados", axis=alt.Axis(title="", format='%'), stack="normalize"),
    tooltip = ["Fecha", "Casos confirmados"],
    color = alt.Color("Sector"),
).properties(
    height = 100,
    width=700
)

alt.vconcat(
    chart1,
    chart2
).configure_concat(
    spacing=0
)

alt.VConcatChart(...)

Fuentes: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19), [Ministerio de Salud](https://www.minsal.cl/), Para la agrupación de comunas en 7 sectores, la página Wikipedia [Comunas de Santiago de Chile](https://es.wikipedia.org/wiki/Anexo:Comunas_de_Santiago_de_Chile#Conurbaci%C3%B3n_de_Santiago)

* No hay datos diarios de nuevos confirmados por comuna. Estos son estimados a través de una interpolación lineal entre las proporciones observadas entre los dos informes más cercanos en el tiempo y aplicados a casos confirmados regionales.